In [1]:
import pandas as pd

In [3]:
train_data = pd.read_csv('../data/train_data.csv')
train_data.head(3)

,label,comment
0,0,国王的工作就是读几句稿子啊
1,0,小朋友看嫌复杂大朋友看想快进的尴尬人物似曾相识唯一的泪点也是复制黏贴
2,0,一个非常丰富传奇的故事拍得这么浅薄大家是怎么给出五星的好奇


In [4]:
test_data = pd.read_csv('../data/test_data.csv')
test_data.head(3)

,comment
0,画面很美但台词实在是太太太矫情不是十几岁少年的正常青涩哪怕故作的沧桑而像没文化的油腻中年对键...
1,这片子好看好看好看好看重要的问题问四遍竟然是前百失望致幻的嗨药是弱者逃避现实的选择你觉得现实...
2,老电影有老电影说不出来的美感即使是近三个小时的电影细节做的仍非精致七武士的故事人类百科


## 分词
- train: n(words)*44068(samples)
- test:  n(words)*9999 (samples)

In [5]:
import jieba 

In [14]:
# 调整分词结果
jieba.suggest_freq("渣男",tune=True)

1

In [17]:
lst_trainWords = [list(jieba.cut(l)) for l in train_data['comment']]

print(len(lst_trainWords), len(lst_trainWords[0]))

44068 8


In [18]:
lst_testWords = [list(jieba.cut(l)) for l in test_data['comment']]

print(len(lst_testWords), len(lst_testWords[0]))

9999 72


## 词向量

In [27]:
from gensim.models import Word2Vec                                         # 词向量
from sklearn.manifold import TSNE                                          # tsne降维as

### 训练词向量

In [28]:
%%time
# 总训集词向量
model_train = Word2Vec(min_count=1, size=50)
model_train.build_vocab(lst_trainWords)

vocab_train = model_train.wv.vocab
lst_trainVecKeys = list(vocab_train.keys())

print(len(lst_trainVecKeys))

62269
Wall time: 2.32 s


In [29]:
%%time
# 目标集词向量
model_test = Word2Vec(min_count=1, size=50)
model_test.build_vocab(lst_testWords)

vocab_test = model_test.wv.vocab
lst_testVecKeys = list(vocab_test.keys())

print(len(lst_testVecKeys))

29136
Wall time: 902 ms


In [30]:
%%time
# 总训集分词set
l_trainWordExt = []
for sen in lst_trainWords:
    l_trainWordExt.extend(sen)
lst_trainWordsSet = set(l_trainWordExt)
print(len(lst_trainWordsSet))

1268514 62269
Wall time: 154 ms


In [31]:
%%time
# 目标集分词set
l_testWordExt = []
for sen in lst_testWords:
    l_testWordExt.extend(sen)
lst_testWordsSet = set(l_testWordExt)
print(len(lst_testWordsSet))

286512 29136
Wall time: 37 ms


In [32]:
assert len(lst_trainWordsSet)==len(lst_trainVecKeys), 'train-words vector is not equal to train-words set'
assert len(lst_testWordsSet)==len(lst_testVecKeys), 'test-words vector is not equal to test-words set'
print('words vector done well')

words vector done well


### 词向量分配

In [34]:
print(len(lst_trainWords), len(lst_trainWords[0]))
print(len(lst_testWords), len(lst_testWords[0]))

44068 8
9999 72


In [38]:
%%time
# 总训集词向量
lst_trainVecSen = []
for sen in lst_trainWords:
    lst_trainVecSen.append([list(model_train.wv.__getitem__(word)) for word in sen])
print(len(lst_trainVecSen)), print(len(lst_trainVecSen[0]))

44068
8
Wall time: 17 s


In [39]:
%%time
# 目标集词向量
lst_testVecSen = []
for sen in lst_testWords:
    lst_testVecSen.append([list(model_test.wv.__getitem__(word)) for word in sen])
print(len(lst_testVecSen)), print(len(lst_testVecSen[0]))

9999
72
Wall time: 4.34 s


In [41]:
assert len(lst_trainVecSen)==len(lst_trainWords), 'train set mapping not success'
assert len(lst_testVecSen)==len(lst_testWords), 'test set mapping not success'
print('all looks good')

all looks good


## 分割数据集

In [43]:
from sklearn.model_selection import train_test_split

In [46]:
X, y = lst_trainVecSen, list(train_data['label'])
print(len(X), len(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(len(X_train), len(X_test), len(y_train), len(y_test))

44068 44068
30847 13221 30847 13221


In [ ]:
## 